# Missing Value Metotları

* **fillna()** Boş ya da NaN değerlerin yerine parantez içindeki değeri atar.


* **dropna()** İçerisinde NaN değer olan satırların tamamını siler. Kalıcı olması için data.drop(**inplace=True**) şeklinde kullanılır.


* Buradaki **NaN** ile veritabanındaki girilmemiş değer olan **Null** aynı mantık

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [3]:
# feather format ile kaydettiğimiz dataframe i okuruz

df = pd.read_feather("tmp/buldozers_1")

In [4]:
# Burada a ile b nin eleman sayısı eşit olmalıdır yoksa dataframe kolonları oluşmaz, kod hata verir.

d = {"a":[1,2,3,4,4,np.nan], "b":[1,2,3,3,4,5]}
toy = pd.DataFrame(d)

In [5]:
toy

,a,b
0,1.0,1
1,2.0,2
2,3.0,3
3,4.0,3
4,4.0,4
5,NaN,5


In [6]:
# NaN değerler için False, dolu değerler için True atar.
toy.isnull()

,a,b
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,True,False


In [7]:
# isnull() metodunu seri üzerinde de kullanabiliriz.
# a serisi için çalıştır
toy["a"].isnull()

0    False
1    False
2    False
3    False
4    False
5     True
Name: a, dtype: bool

In [8]:
# Yukarıdakiyle aynı sonucu verir
pd.isnull(toy["a"])

0    False
1    False
2    False
3    False
4    False
5     True
Name: a, dtype: bool

* İlgili column içerisindeki **toplam missing value(NaN)** sayısını aşağıda **1** old. göstermektedir.

In [9]:
# Bir serinin içerisinde NaN değer olup olmadığını anlarız.
# Null değerler olmasaydı sonuç 0(sıfır) olurdu.

toy["a"].isnull().sum()

1

In [10]:
def fix_missing(df, col, name):
    
    #eğer bu numeric type ise:
    #bunu belirtmemizin sebebi string typelarda(kategorik) "-1" değeri NaN yerine gelecekti ve model onun NaN old. anlayacaktı
    #Burda ise numeric veri tipi old biz onun aslında NaN old açıklarız
    if is_numeric_dtype(col):
        
        #hangi kolona bakıyorsan onun sum() ını bul. sum() sıfıra eşitse columnda NaN yok, sıfırdan farklıysa if in içerisine gir 
        if pd.isnull(col).sum:       
            
            df[name+"_na"] = pd.isnull(col)  #o column a "columnadı_na" yaz, tüm column u pd.isnull column ile değiştir
        df[name] = col.fillna(col.median)    #column içerisindeki NaN ları column un ort değeriyle değiştir
            
    

* Yukarıdaki fonksiyonu aşağıdaki döngüyle test edilir

In [11]:
for n,c in df.items():
        if is_numeric_dtype(c):
            if df[n].isnull().sum():
                print(n)

auctioneerID
MachineHoursCurrentMeter


* Burada yazan **auctioneerID** ve **MachineHoursCurrentMeter** columnları hem numeric tipte olup hem de içerisinde **NaN** değerler bulunan columnlar.

In [12]:
# ilgili column içerisindeki toplam NaN değer sayısını verir

df["MachineHoursCurrentMeter"].isnull().sum()

265194

In [13]:
fix_missing(df,df["MachineHoursCurrentMeter"], "MachineHoursCurrentMeter" )

* Yukarıdaki fonksiyonu uyguladığımda **missing value lar** gider.

In [14]:
df["MachineHoursCurrentMeter"].isnull().sum()

0

* Missing value lar yerine gelen column median değerleri

In [15]:
df["MachineHoursCurrentMeter"]

0                                                        68
1                                                      4640
2                                                      2838
3                                                      3486
4                                                       722
                                ...                        
412693    <bound method Series.median of 0           68....
412694    <bound method Series.median of 0           68....
412695    <bound method Series.median of 0           68....
412696    <bound method Series.median of 0           68....
412697    <bound method Series.median of 0           68....
Name: MachineHoursCurrentMeter, Length: 412698, dtype: object

* Fonksiyonun **NaN değerler için** oluşturduğu yeni "**MachineHoursCurrentMeter_na** column u "

In [16]:
df["MachineHoursCurrentMeter_na"]

0         False
1         False
2         False
3         False
4         False
          ...  
412693     True
412694     True
412695     True
412696     True
412697     True
Name: MachineHoursCurrentMeter_na, Length: 412698, dtype: bool